In [ ]:
import os
import pickle
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands

# Hands 모델 초기화
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# 데이터 디렉토리 경로
DATA_DIR = '../data'

# 데이터와 라벨을 저장할 리스트 초기화
data = []
labels = []

# 데이터 디렉토리 내의 각 디렉토리에 대해 처리
for dir_ in os.listdir(DATA_DIR):
    # 디렉토리만 처리
    if os.path.isdir(os.path.join(DATA_DIR, dir_)):
        dir_path = os.path.join(DATA_DIR, dir_)
        
        # 현재 폴더 이름이 숫자라면 int로 변환
        try:
            dir_num = int(dir_)
        except ValueError:
            dir_num = None

        # 디렉토리 내의 각 이미지 파일에 대해 처리
        for img_path in os.listdir(dir_path):
            # 이미지 파일인지 확인
            if img_path.endswith(('.jpg', '.jpeg', '.png')):
                # 이미지 읽기
                img = cv2.imread(os.path.join(dir_path, img_path))
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                # 손 위치 검출
                results = hands.process(img_rgb)
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        data_aux = []
                        for i in range(len(hand_landmarks.landmark) - 1):
                            # 각 관절 간 벡터 계산
                            x1, y1, z1 = hand_landmarks.landmark[i].x, hand_landmarks.landmark[i].y, hand_landmarks.landmark[i].z
                            x2, y2, z2 = hand_landmarks.landmark[i + 1].x, hand_landmarks.landmark[i + 1].y, hand_landmarks.landmark[i + 1].z

                            vx = x2 - x1
                            vy = y2 - y1
                            vz = z2 - z1

                            # 벡터 정규화
                            norm = np.sqrt(vx**2 + vy**2 + vz**2)
                            vx /= norm
                            vy /= norm
                            vz /= norm

                            data_aux.extend([vx, vy, vz])

                        # 0~30 폴더: 데이터 길이 60인지 확인
                        if dir_num is not None and 0 <= dir_num <= 30:
                            if len(data_aux) == 60:
                                data.append(data_aux)
                                labels.append(dir_)
                            else:
                                print(f"Skipping file {img_path} in {dir_}: unexpected data length {len(data_aux)}")
                        # 그 외 폴더: 인식된 모든 손 데이터 추가
                        else:
                            data.append(data_aux)
                            labels.append(dir_)

# 데이터를 pickle 파일로 저장
with open('data.pickle', 'wb') as f:
    pickle.dump({'data': data, 'labels': labels}, f)

print("Dataset created successfully.")